<a href="https://colab.research.google.com/github/macscheffer/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/Copy_of_DS_Unit_4_Sprint_3_Neural_Nets_Spring_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

A neuron receives information from previous nodes in the network, transforms that input into an output that it then sends on to other nodes with an activation function.

An input layer is the first layer in a standard feed forward neural network, ie it is being fed information from the initial features of the data.

Hidden layer is any layer between the input layer and output layer, kind of like the way the question is formatted. Hidden layers are what make neural networks so powerful when trying to find nonlinear relationships.

the output layer transforms information from the final hidden layer into output that is the prediction.

Activation is a way to filter what data gets passed from layer to layer in the neural network. 

Backpropagation is the messenger telling the network whether or not the network made a mistake when it made a prediction. It is the algorithm that updates weights and bias parameters of neural networks. 

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [0]:
import numpy as np
import pandas as pd

In [0]:
class Perceptron():
  
  def __init__(self, learning_rate=0.1, iterations=100, tolerance=0.000001):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.tolerance = tolerance
        
  
  def fit(self, X, y):
        # initialize weights
        self.weights_ = np.random.uniform(-0.01, 0.01, X.shape[1] + 1)
        
        self.costs_ = []
        for i in range(self.iterations):
            preds = self.predict_proba(X)
            errors = preds - y
            cost = np.sum(errors ** 2)
            self.costs_.append(cost)
            gradient = np.dot(X.T, errors)
            self.weights_[1:] -= self.learning_rate * gradient
            self.weights_[0] -= np.mean(errors)
 
        return self
      
  def predict_proba(self, X):
    return 1. / (1. + np.exp(-(np.dot(X, self.weights_[1:]) + self.weights_[0])))
      
  def predict(self, X):
    return np.where(self.predict_proba(X)>=0.5, 1, 0)

In [4]:
X = np.array([[1,0,1],
            [1,1,1],
            [0,0,1],
            [0,1,1]])
y = np.array([1,1,0,0])

perc = Perceptron()
perc.fit(X, y)
perc.predict(X)

array([1, 1, 0, 0])

## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')

In [13]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [12]:
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['target'])
y = df['target']


scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape(-1,13)
y = np.array(y).reshape(-1,1)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [0]:
class NeuralNetwork(object):
  def __init__(self):
    self.inputs = 13
    self.hiddenNodes = 7
    self.outputNodes = 1

    # Initialize Weights:
    self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) 
    self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes)
    
    
  def feed_forward(self, X):
    
    self.hidden_sum = np.dot(X, self.L1_weights)
    self.activated_hidden = self.sigmoid(self.hidden_sum)
    self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
    self.activated_output = self.sigmoid(self.output_sum)
    
    return self.activated_output
    
  def sigmoid(self, s):
    return 1/(1+np.exp(-s))
  
  
  def sigmoidPrime(self, s):
    return s * (1 - s)
  
  
  def backward(self, X, y, output):
    self.output_error = y - output
    
    self.output_delta = self.output_error * self.sigmoidPrime(output) 
    
    self.z2_error = self.output_delta.dot(self.L2_weights.T)
    
    self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden) 
    
    self.L1_weights += X.T.dot(self.z2_delta) 

    self.L2_weights += self.activated_hidden.T.dot(self.output_delta) 
    
  def train (self, X, y):
    output = self.feed_forward(X)
    self.backward(X, y, output)
    

In [20]:
nn = NeuralNetwork()
for i in range(1000): 
  if i+1 in [1,2,3,4,5] or (i+1) % 5 == 0:
    print('EPOCH', i+1)   
    print("Loss:" + str(np.mean(np.square(y - nn.feed_forward(X))))) # mean sum squared loss
    print("\n")
  nn.train(X, y)

EPOCH 1
Loss:0.4534137786347697


EPOCH 2
Loss:0.4554282424419276


EPOCH 3
Loss:0.45542758324679405


EPOCH 4
Loss:0.4554268574112785


EPOCH 5
Loss:0.45542605435603867


EPOCH 10
Loss:0.4554203064755986


EPOCH 15
Loss:0.45540836041407157


EPOCH 20
Loss:0.45537350765551265


EPOCH 25
Loss:0.45517759983494843


EPOCH 30
Loss:0.44965445000995297


EPOCH 35
Loss:0.4369969998515835


EPOCH 40
Loss:0.270539826360468


EPOCH 45
Loss:0.1561323644455471


EPOCH 50
Loss:0.1485655545751396


EPOCH 55
Loss:0.1474686854339892


EPOCH 60
Loss:0.14535264150199675


EPOCH 65
Loss:0.13799153710082607


EPOCH 70
Loss:0.13091284202782674


EPOCH 75
Loss:0.12299070278336763


EPOCH 80
Loss:0.11486872235285266


EPOCH 85
Loss:0.10350591324458301


EPOCH 90
Loss:0.09801089107164311


EPOCH 95
Loss:0.09512950902469022


EPOCH 100
Loss:0.09404588877922151


EPOCH 105
Loss:0.0939190983477646


EPOCH 110
Loss:0.09382574933681245


EPOCH 115
Loss:0.09374687346116428


EPOCH 120
Loss:0.093670035989256


EPOCH

## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense

def baseline(optimizer='adam', init='uniform', activation='relu'):
  model = Sequential()
  model.add(Dense(13, input_dim=13, activation=activation))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

model = KerasClassifier(build_fn=baseline, epochs=100, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
193/193 [==============================] - 1s 3ms/step - loss: 0.5755 - acc: 0.7098
Epoch 2/100
193/193 [==============================] - 0s 265us/step - loss: 0.5185 - acc: 0.7617
Epoch 3/100
193/193 [==============================] - 0s 257us/step - loss: 0.4779 - acc: 0.7824
Epoch 4/100
193/193 [==============================] - 0s 271us/step - loss: 0.4473 - acc: 0.8083
Epoch 5/100
193/193 [==============================] - 0s 248us/step - loss: 0.4213 - acc: 0.8290
Epoch 6/100
193/193 [==============================] - 0s 246us/step - loss: 0.4013 - acc: 0.8653
Epoch 7/100
193/193 [==============================] - 0s 265us/step - loss: 0.3847 - acc: 0.8705
Epoch 8/100
193/193 [==============================] - 0s 277us/step - loss: 0.3705 - acc: 0.8756
Epoch 9/100
193/193 [==============================] - 0s 254us/step - loss: 0.3590 - acc: 0.8705
Epoch 10/100
193/193 [==============================] - 0s 253us/step - loss: 0.3488 - acc: 0.8705
Epoch 11/100
193/193 

In [0]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import SGD, Adam, Nadam
from keras.wrappers.scikit_learn import KerasClassifier

In [0]:
model = KerasClassifier(build_fn=baseline, 
                               epochs=5,
                               batch_size=10,
                               verbose=1)


In [0]:

param_grid = {}
# checking that this works
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=5,
                    n_jobs=1)
grid_result = grid.fit(X_train, y_train)


Epoch 1/5
193/193 [==============================] - 5s 28ms/step - loss: 0.6921 - acc: 0.5544
Epoch 2/5
193/193 [==============================] - 0s 250us/step - loss: 0.6432 - acc: 0.6166
Epoch 3/5
193/193 [==============================] - 0s 262us/step - loss: 0.6018 - acc: 0.6684
Epoch 4/5
193/193 [==============================] - 0s 250us/step - loss: 0.5680 - acc: 0.7306
Epoch 5/5
193/193 [==============================] - 0s 189us/step
Epoch 1/5
193/193 [==============================] - 5s 27ms/step - loss: 0.6225 - acc: 0.6321
Epoch 2/5
193/193 [==============================] - 0s 255us/step - loss: 0.5782 - acc: 0.7254
Epoch 3/5
193/193 [==============================] - 0s 261us/step - loss: 0.5451 - acc: 0.7513
Epoch 4/5
193/193 [==============================] - 0s 252us/step - loss: 0.5140 - acc: 0.7668
Epoch 5/5
193/193 [==============================] - 0s 205us/step
Epoch 1/5
194/194 [==============================] - 5s 27ms/step - loss: 0.7560 - acc: 0.4794
Epoch

In [0]:
grid_result

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fcd2f1ff3c8>,
       fit_params=None, iid='warn', n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [0]:
grid_result.best_estimator_

In [0]:
grid_result.best_params_
# i just checked how the grid was working so this is expected

{}

In [0]:
grid_result.best_score_

0.7644628069617532

In [0]:
# need to reset my model building stage. 

def create_model(dense_layers=2,
                 dense_nodes=5,
                 dropout=False,
                 dropout_pct=0.0,
                 activation='sigmoid',
                 weight_initializer='glorot_uniform',
                 optimizer=SGD,
                 lr=0.0001,
                 input_shape=(X_train.shape[1],)):
    
    model = Sequential()
   
    model.add(Dense(dense_nodes, 
                    input_shape=input_shape,
                    kernel_initializer=weight_initializer,
                    activation=activation))
    for _ in range(dense_layers):
        model.add(Dense(dense_nodes,
                        kernel_initializer=weight_initializer,
                        activation=activation))
        if dropout:
            model.add(Dropout(rate=dropout_pct))

    model.add(Dense(1, activation='sigmoid'))
    
    optimizer=optimizer(lr=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
              
    return model

In [0]:

param_grid = {'batch_size': [10, 80],
              'epochs': [20,90]}

model = KerasClassifier(build_fn=create_model, 
                               epochs=20,
                               batch_size=10,
                               verbose=10)

grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=3,
                    n_jobs=1)
grid_result = grid.fit(X_train, y_train)

Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/90
Epoch 2/90
Epoch 3/90
Epoch 4/90
Epoch 5/90
Epoch 6/90
Epoch 7/90
Epoch 8/90
Epoch 9/90
Epoch 10/90
Epoch 11/90
Epoch 12/90
Epoch 13/90
Epoch 14/90
Epoch 15/90
Epoch 16/90
Epoch 17/90
Epoch 18/90
Epoch 19/90
Epoch 20/90
Epoch 21/90
Epoch 22/90
Epoch 23/90
Epoch 24/90
Epoch 25/90
Epoch 26/90
Epoc

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/90
Epoch 2/90
Epoch 3/90
Epoch 4/90
Epoch 5/90
Epoch 6/90
Epoch 7/90
Epoch 8/90
Epoch 9/90
Epoch 10/90
Epoch 11/90
Epoch 12/90
Epoch 13/90
Epoch 14/90
Epoch 15/90
Epoch 16/90
Epoch 17/90
Epoch 18/90
Epoch 19/90
Epoch 20/90
Epoch 21/90
Epoch 22/90
Epoch 23/90
Epoch 24/90
Epoch 25/90
Epoch 26/90
Epoch 27/90
Epoch 28/90
Epoch 29/90
Epoch 30/90
Epoch 31/90
Epoch 32/90
Epoch 33/90
Epoch 34/90
Epoch 35/90
Epoch 36/90
Epoch 37/90
Epoch 38/90
Epoch 39/90
Epoch 40/90
Epoch 41/90
Epoch 42/90
Epoch 43/90
Epoch 44/90
Epoch 45/90
Epoch 46/90
Epoch 47/90
Epoch 48/90
Epoch 49/90
Epoch 50/90
Epoch 51/90
Epoch 52/90
Epoch 53/90
Epoch 54/90
Epoch 55/90
Epoch 56/90
Epoch 57/90
Epoch 58/90
Epoch 59/90
Epoch 60/90
Epoch 61/90
Epoch 62/90
Epoch 63/90
Epoch 64/90
Epoch 65/90
Epoch 66/90
Epoch 67/90
Epoch 68/90
Epoch 69/90
Epoch 70/90
Epoch 71/90
Epoch 72/90
Epoch 73/90
Epoch 74/90
Epoch 75/90
Epoch 76/90
Epoch 77/90
Epoch 78/90
Epoch 79/90
Epoch 80/90
Epoch 81/90
Epoch 82/90
Epoch 83/90
Epoch 84/90
E

In [0]:
grid_result

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fcd26116d68>,
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'batch_size': [10, 80], 'epochs': [20, 90]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [0]:
grid_result.best_estimator_

In [0]:
grid_result.best_params_

{'batch_size': 10, 'epochs': 90}

In [0]:
grid_result.best_score_

0.5413223211922922

In [0]:
# terrible grid search. just using my initial defaults was better.